In [ ]:
import tensorflow as tf
#import tensorflow.contrib.learn.python.learn as learn
import tflearn 
import scipy as sp
import numpy as np 
import matplotlib.pyplot as plt
from random import shuffle, randint
from sklearn.utils import shuffle as mutualShuf
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score
import os
import pandas as pd
import sklearn
import datetime
%matplotlib inline

# Import and process data

In [ ]:
def processClassData(classData):
    """
    Process classData.
    
    Returns a one-hot array of shape [len(classData), 2].
    """
    # Convert label data to one-hot array
          
    classDataOH = np.zeros((len(classData),2))
    classDataOH[np.arange(len(classData)), classData] = 1
    
    return classDataOH

unseenData = np.load("/tmp/unseenData.npy")
unseenClass = np.load("/tmp/unseenClass.npy")
ecgData = np.load("/tmp/ecgData.npy")
classData = np.load("/tmp/classData.npy")
unseenClassOH = processClassData(unseenClass)
classDataOH = processClassData(classData)

In [ ]:
print(ecgData[1].shape)
plt.plot(ecgData[1])

In [ ]:
print(unseenData.shape)
plt.plot(unseenData[0])

In [ ]:
def splitData(coilData, classData):
    """
    Split data into healthy and ill types.
    """
    illData = []
    healthData = []
    
    for index, item in enumerate(classData):
        if item == 1:
            illData.append(coilData[index])
        if item == 0:
            healthData.append(coilData[index])
            
    return illData, healthData

tillarr, thealtharr = splitData(unseenData, unseenClass)

# Neural Network

In [ ]:
spec = []
sens = []
auc = []

In [ ]:
for i in np.arange(3):
    for j in np.arange(33):
        sess = tf.InteractiveSession()
        tf.reset_default_graph()
        tflearn.initializations.normal()

        # Input layer:
        net = tflearn.layers.core.input_data(shape=[None, 200, 1])

        # First layer:
        net = tflearn.layers.conv.conv_1d(net, 32, 5, activation="leaky_relu")
        net = tflearn.layers.conv.max_pool_1d(net, 2, strides=2)

        # Second layer:
        net = tflearn.layers.conv.conv_1d(net, 64, 5, activation="leaky_relu")
        net = tflearn.layers.conv.max_pool_1d(net, 2, strides=2)

        # Fully connected layer 1:
        net = tflearn.layers.core.fully_connected(net, 1024, regularizer="L2", weight_decay=0.001, activation="leaky_relu")

        # Dropout layer:
        net = tflearn.layers.core.dropout(net, keep_prob=0.5)

        # Output layer:
        net = tflearn.layers.core.fully_connected(net, 2, activation="softmax")

        net = tflearn.layers.estimator.regression(net, optimizer='adam', loss='categorical_crossentropy',\
                                 learning_rate=0.00001)

        model = tflearn.DNN(net, tensorboard_verbose=0)
        dummyData = ecgData[:,0+25*j:200+25*j]
        model.fit(dummyData, classDataOH, n_epoch=1, show_metric=True)

        sens.append(model.evaluate(np.array(thealtharr)[:,0+25*j:200+25*j], np.tile([1,0], (len(thealtharr), 1)))[0])
        spec.append(model.evaluate(np.array(tillarr)[:,0+25*j:200+25*j], np.tile([0,1], (len(tillarr), 1)))[0])
        lenh = len(thealtharr)
        leni = len(tillarr)
        predicted = np.array(model.predict(np.array(unseenData)[:,0+25*j:200+25*j]))
        auc.append(roc_auc_score(unseenClass, predicted[:,1]))

In [ ]:
# Save model?
#now = datetime.datetime.now()
#model.save("./outData/models/cleanECG_undiff_FrankCAT2_"+now.isoformat()+"_.tflearn")

In [ ]:
avg = []
for i in np.arange(np.shape(spec)[0]):
    avg.append(np.mean([spec[i], sens[i]]))

In [ ]:
auc = np.reshape(auc, (3,-1))
print(auc.shape)
print(np.arange(100,925,25).shape)
plt.plot(np.arange(100,925,25), auc[0], "x")
plt.plot(np.arange(100,925,25), auc[1], "x")
plt.plot(np.arange(100,925,25), auc[2], "x")
plt.plot(ecgData[0]*1.5)
plt.title("ROC AUC values for sections of ECG scans \n 200 wide sections, step 25")
plt.xlabel("Index")
plt.ylabel("AUC")
plt.tight_layout()
plt.savefig("../thesis/images/ecgauc.pdf")